## KOSIS API 테스트

- 본격적 수집 이전에 하나 가져와서 처리해보는 식으로

In [ ]:
import time
from tqdm import tqdm
import requests

import pandas as pd
# import pandas.io.json._normalize as json_normalize

In [ ]:
# ## 통계표선택에서 url 가져옴 > 테스트용 url임 ##

# url = "https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T00+T10+T11+T12+T13+T14+T15+T19+T20+T21+T22+T23+T24+T25+T26+T27+T28+T29+T30+T31+T32+T33+T39+T40+T50+T60+&objL1=00+04+05+03+11+21+22+23+24+25+26+29+31+32+33+34+35+36+37+38+39+&objL2=0+1+2+&objL3=000+019+020+025+030+035+040+045+050+055+060+065+070+075+080+085+086+&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2020&endPrdDe=2020&loadGubun=2&orgId=101&tblId=DT_1IN1509"
url = "https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T3+&objL1=ALL&objL2=ALL&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=H&startPrdDe=202102&endPrdDe=202102&loadGubun=2&orgId=101&tblId=DT_1ES3C04S"

ti = time.time()
res = requests.get(url)
print("elapsed:", time.time() - ti)

data = res.json()

elapsed: 5.185199499130249


In [ ]:
## 데이터프레임 생성 ##
df = pd.json_normalize(data)

In [ ]:
## 컬럼 1차 필터링 ##
cols = [col for col in df.columns if "ENG" not in col]  # 영문컬럼 삭제
df = df.loc[:, cols]

In [ ]:
## 컬럼명 수정 ##

renames = {}

# 통계표명
renames["TBL_NM"] = "통계표"

# 시점 관련
renames["PRD_DE"] = "시점"
renames["PRD_SE"] = "주기"

# 데이터 및 분류 관련
renames["DT"] = "값"
renames["ITM_NM"] = "항목명"
renames["UNIT_NM"] = "단위"


# 함수
def get_col_single_value(df, col_name):
    """단일 값밖에 없는 컬럼의 값을 문자열로 반환한다. (set과 join 이용)
    """
    return "".join(set(df[col_name].to_list()))


# 카테고리들 컬럼명 변경
cat_obj_cols = [col for col in df.columns if "C" in col and "OBJ" in col]
for cat_obj in cat_obj_cols:
    origin_name = cat_obj.replace("_OBJ", "")  #  OBJ 뺀 컬럼 이름
    new_name = get_col_single_value(df, cat_obj)
    renames[origin_name] = new_name
    origin_code_name = origin_name.replace("_NM", "")
    new_code_name = f"{new_name}_코드"
    renames[origin_code_name] = new_code_name

df.rename(renames, axis=1, inplace=True)

In [ ]:
# ## 컬럼명 수정 후, 수정한 컬럼들만 남기기 ##
last_cols = ["항목명", "값", "단위"]
cols = [col for col in renames.values() if col not in last_cols]
df = df.loc[:, cols + last_cols]

In [ ]:
df
### 여기까지 작업한 뒤에 csv로 임시로 저장해두고 불러와서 다음 작업 하면 될 듯 ###

,통계표,시점,주기,행정구역별(시군),행정구역별(시군)_코드,종사상지위별,종사상지위별_코드,항목명,값,단위
0,시군구/종사상지위별 취업자(근무지기준),202102,S,서울 종로구,1101,계,00,취업자 (천명),278.9,천명
1,시군구/종사상지위별 취업자(근무지기준),202102,S,서울 종로구,1101,임금근로자,10,취업자 (천명),240.8,천명
2,시군구/종사상지위별 취업자(근무지기준),202102,S,서울 종로구,1101,- 상용근로자,11,취업자 (천명),202.5,천명
3,시군구/종사상지위별 취업자(근무지기준),202102,S,서울 종로구,1101,- 임시일용근로자,12,취업자 (천명),38.3,천명
4,시군구/종사상지위별 취업자(근무지기준),202102,S,서울 종로구,1101,비임금근로자,20,취업자 (천명),38.1,천명
...,...,...,...,...,...,...,...,...,...,...
1135,시군구/종사상지위별 취업자(근무지기준),202102,S,서귀포시,3902,계,00,취업자 (천명),118,천명
1136,시군구/종사상지위별 취업자(근무지기준),202102,S,서귀포시,3902,임금근로자,10,취업자 (천명),65.9,천명
1137,시군구/종사상지위별 취업자(근무지기준),202102,S,서귀포시,3902,- 상용근로자,11,취업자 (천명),45.6,천명
1138,시군구/종사상지위별 취업자(근무지기준),202102,S,서귀포시,3902,- 임시일용근로자,12,취업자 (천명),20.3,천명


### 함수화

- url만 넣으면 위 작업 해서 반환하도록 함수제작

In [ ]:
def get_kosis_api_data(url, print_elapsed=True):
    """ '통계표선택'을 통해 생성한 API 요청 url을 받고, 데이터를 받아와 정리 후 통계표명과 데이터프레임으로 반환한다.
    """
    ## 요청-반환 ##
    ti = time.time()
    res = requests.get(url)
    if print_elapsed:
        print("API GET Request Elapsed:", round(time.time() - ti, 2))

    ## 데이터프레임 생성 ##
    data = res.json()
    df = pd.io.json.json_normalize(data)

    ## 컬럼 1차 필터링 ##
    cols = [col for col in df.columns if "ENG" not in col]  # 영문컬럼 삭제
    df = df.loc[:, cols]

    ## 컬럼명 수정 ##
    renames = {}

    # 통계표명
    renames["TBL_NM"] = "통계표"
    # 시점 관련
    renames["PRD_DE"] = "시점"
    renames["PRD_SE"] = "주기_코드"
    # 데이터 및 분류 관련
    renames["DT"] = "값"
    renames["ITM_NM"] = "항목명"
    renames["UNIT_NM"] = "단위"

    # 함수
    get_col_single_value = lambda df, col_name: "".join(set(df[col_name].to_list()))  # 해당 컬럼의 값들을 중복 제거하고 하나의 문자열로 반환

    # 카테고리들 컬럼명 변경 (함수 이용)
    cat_obj_cols = [col for col in df.columns if "C" in col and "OBJ" in col]
    for cat_obj in cat_obj_cols:
        origin_name = cat_obj.replace("_OBJ", "")  #  OBJ 뺀 컬럼 이름
        new_name = get_col_single_value(df, cat_obj)
        renames[origin_name] = new_name

        origin_code_name = origin_name.replace("_NM", "")
        new_code_name = f"{new_name}_코드"
        renames[origin_code_name] = new_code_name

    # 컬럼명 수정
    df.rename(renames, axis=1, inplace=True)

    ## 컬럼명 수정 후, 수정한 컬럼들만 남기기 ##
    last_cols = ["항목명", "값", "단위"]
    cols = [col for col in renames.values() if col not in last_cols]
    df = df.loc[:, cols + last_cols]

    ## 반환 ##
    stat_name = get_col_single_value(df, "통계표")

    return stat_name, df


def collect_kosis_api_data_with_date_range(url_base, date_cycle="연간", start_year=2019, end_year=2021):
    """kosis api를 통해 기간의 데이터를 수집함

    Args:
        url_base(str): 통계표선택에서 날짜 지정 수집으로 가져온 url에서 날짜 부분을 {date}로 변경한 것 (format을 이용해서 date를 지정함)
        ~

    Returns:
        저장할 때 쓸 수 있는 파일명(csv)과 데이터프레임 반환
    """
    # date_cycle 유형에 따라 날짜를 만들기 (date_cycle, start_yr, end_yr 이용)
    if date_cycle == "연간":
        elements = ["",]
    elif date_cycle == "반기":
        elements = ["01", "02"]
    elif date_cycle == "월간":
        elements = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

    dates = []
    for yr in range(start_year, end_year+1):
        for ele in elements:
            dates.append(f"{yr}{ele}")

    # 실행
    dfs = []
    for d in tqdm(dates):
        # url에서 startPreDe와 endPreDe를 d로 수정 (={date}로 바꿔놓은것 format으로 넣기)
        name, df = get_kosis_api_data(url_base.format(date=d), print_elapsed=False)  # 날짜만 다르게 해서 수집하는 것
        dfs.append(df)

    # 데이터프레임 병합
    filename = f"{name}_{date_cycle}_{dates[0]}-{dates[-1]}.csv"  # 저장하려면 이 파일명으로 저장하기
    result = pd.concat(dfs)
    result["주기"] = date_cycle
    result.reset_index(drop=True, inplace=True)

    return filename, result

In [ ]:
## 테스트 - 단일 수집기 ##
url = "https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T3+&objL1=ALL&objL2=ALL&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=H&startPrdDe=202102&endPrdDe=202102&loadGubun=2&orgId=101&tblId=DT_1ES3C04S"
# url = "https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T00+T10+T11+T12+T13+T14+T15+T19+T20+T21+T22+T23+T24+T25+T26+T27+T28+T29+T30+T31+T32+T33+T39+T40+T50+T60+&objL1=00+04+05+03+11+21+22+23+24+25+26+29+31+32+33+34+35+36+37+38+39+&objL2=0+1+2+&objL3=000+019+020+025+030+035+040+045+050+055+060+065+070+075+080+085+086+&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2020&endPrdDe=2020&loadGubun=2&orgId=101&tblId=DT_1IN1509"

name, df = get_kosis_api_data(url)
print("-" * 50)
print(name)  # 이렇게 반환받은 이름에 추가 정보 (시점 등) 추가해서 하기? (날짜 목록들 넣고 그거 기반으로 하는것?)
print("-" * 50)
df

API GET Request Elapsed: 1.37
--------------------------------------------------
시군구/종사상지위별 취업자(근무지기준)
--------------------------------------------------


,통계표,시점,주기_코드,행정구역별(시군),행정구역별(시군)_코드,종사상지위별,종사상지위별_코드,항목명,값,단위
0,시군구/종사상지위별 취업자(근무지기준),202102,S,서울 종로구,1101,계,00,취업자 (천명),278.9,천명
1,시군구/종사상지위별 취업자(근무지기준),202102,S,서울 종로구,1101,임금근로자,10,취업자 (천명),240.8,천명
2,시군구/종사상지위별 취업자(근무지기준),202102,S,서울 종로구,1101,- 상용근로자,11,취업자 (천명),202.5,천명
3,시군구/종사상지위별 취업자(근무지기준),202102,S,서울 종로구,1101,- 임시일용근로자,12,취업자 (천명),38.3,천명
4,시군구/종사상지위별 취업자(근무지기준),202102,S,서울 종로구,1101,비임금근로자,20,취업자 (천명),38.1,천명
...,...,...,...,...,...,...,...,...,...,...
1135,시군구/종사상지위별 취업자(근무지기준),202102,S,서귀포시,3902,계,00,취업자 (천명),118,천명
1136,시군구/종사상지위별 취업자(근무지기준),202102,S,서귀포시,3902,임금근로자,10,취업자 (천명),65.9,천명
1137,시군구/종사상지위별 취업자(근무지기준),202102,S,서귀포시,3902,- 상용근로자,11,취업자 (천명),45.6,천명
1138,시군구/종사상지위별 취업자(근무지기준),202102,S,서귀포시,3902,- 임시일용근로자,12,취업자 (천명),20.3,천명


In [ ]:
## 테스트 - 기간 수집기 ##

## 짧은거
# url_base = "https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T3+&objL1=ALL&objL2=ALL&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=H&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1ES3C04S"
# filename, df = collect_kosis_api_data_with_date_range(url_base, "반기", 2019, 2021)

## 긴거
url_base = "https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=M2MzOGYwMTQyODlmMWRjOGRkYzQ3YzA5ZTQxZTExNTI=&itmId=T20+T21+T22+&objL1=00+11+11110+11140+11170+11200+11215+11230+11260+11290+11305+11320+11350+11380+11410+11440+11470+11500+11530+11545+11560+11590+11620+11650+11680+11710+11740+26+26110+26140+26170+26200+26230+26260+26290+26320+26350+26380+26410+26440+26470+26500+26530+26710+27+27110+27140+27170+27200+27230+27260+27290+27710+28+28110+28114+28118+28140+28170+28177+28185+28200+28230+28237+28245+28260+28265+28710+28720+29+29110+29140+29155+29170+29200+30+30110+30140+30170+30200+30230+31+31110+31140+31170+31200+31710+36+36110+41+41110+41111+41113+41115+41117+41130+41131+41133+41135+41150+41170+41171+41173+41190+41195+41197+41199+41210+41220+41222+41224+41230+41250+41270+41271+41273+41280+41281+41283+41285+41287+41290+41310+41350+41360+41362+41370+41390+41410+41430+41450+41460+41461+41463+41465+41480+41500+41550+41570+41590+41592+41594+41610+41630+41650+41670+41710+41720+41730+41740+41750+41770+41790+41800+41810+41820+41830+41840+41850+41860+41870+41880+41890+42+42110+42130+42150+42170+42190+42210+42230+42715+42720+42730+42745+42750+42760+42770+42780+42790+42800+42810+42820+42830+42840+42850+43+43110+43112+43114+43111+43113+43115+43116+43130+43150+43710+43720+43730+43740+43745+43750+43760+43770+43780+43795+43800+43950+44+44270+44130+44131+44133+44150+44170+44180+44190+44200+44210+44220+44230+44250+44710+44730+44740+44750+44760+44770+44780+44790+44800+44810+44820+44825+44830+44840+44860+45+45110+45111+45113+45118+45130+45140+45145+45180+45190+45210+45710+45720+45730+45740+45750+45760+45770+45780+45790+45800+45810+45820+45830+46+46110+46130+46150+46170+46190+46210+46230+46710+46720+46730+46740+46750+46760+46770+46780+46790+46800+46810+46820+46830+46840+46850+46860+46870+46880+46890+46900+46910+47+47110+47111+47113+47130+47150+47170+47190+47210+47230+47250+47270+47280+47290+47710+47720+47730+47740+47750+47760+47770+47780+47795+47800+47810+47820+47830+47840+47850+47860+47870+47880+47890+47900+47910+47920+47930+47940+48+48120+48121+48123+48125+48127+48129+48110+48130+48151+48153+48160+48170+48190+48210+48220+48230+48240+48245+48250+48252+48270+48290+48310+48330+48332+48710+48720+48730+48740+48750+48760+48775+48780+48795+48800+48810+48820+48830+48840+48850+48860+48870+48880+48890+50+50110+50130+50910+50920+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={date}&endPrdDe={date}&loadGubun=2&orgId=101&tblId=DT_1B040A3"
filename, df = collect_kosis_api_data_with_date_range(url_base, "연간", 2019, 2021)

print("-" * 50)
print(filename)
print("-" * 50)
df.info()  # '값'의 자료형 변경해서 사용해야 함
print("-" * 50)
df

100%|██████████| 3/3 [00:05<00:00,  1.95s/it]

--------------------------------------------------
행정구역(시군구)별 성별 인구수_연간_2019-2021.csv
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550 entries, 0 to 2549
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   통계표            2550 non-null   object
 1   시점             2550 non-null   object
 2   주기_코드          2550 non-null   object
 3   행정구역(시군구)별     2550 non-null   object
 4   행정구역(시군구)별_코드  2550 non-null   object
 5   항목명            2550 non-null   object
 6   값              2550 non-null   object
 7   단위             2550 non-null   object
 8   주기             2550 non-null   object
dtypes: object(9)
memory usage: 179.4+ KB
--------------------------------------------------


,통계표,시점,주기_코드,행정구역(시군구)별,행정구역(시군구)별_코드,항목명,값,단위,주기
0,행정구역(시군구)별 성별 인구수,2019,A,전국,00,총인구수,51849861,명,연간
1,행정구역(시군구)별 성별 인구수,2019,A,전국,00,남자인구수,25864816,명,연간
2,행정구역(시군구)별 성별 인구수,2019,A,전국,00,여자인구수,25985045,명,연간
3,행정구역(시군구)별 성별 인구수,2019,A,서울특별시,11,총인구수,9729107,명,연간
4,행정구역(시군구)별 성별 인구수,2019,A,서울특별시,11,남자인구수,4744059,명,연간
...,...,...,...,...,...,...,...,...,...
2545,행정구역(시군구)별 성별 인구수,2021,A,제주시,50110,남자인구수,246494,명,연간
2546,행정구역(시군구)별 성별 인구수,2021,A,제주시,50110,여자인구수,246602,명,연간
2547,행정구역(시군구)별 성별 인구수,2021,A,서귀포시,50130,총인구수,183663,명,연간
2548,행정구역(시군구)별 성별 인구수,2021,A,서귀포시,50130,남자인구수,92683,명,연간


In [ ]:
# 자료형 변경 테스트
def to_float(x, to_none=["-"]):
    if x in to_none:
        return None
    else:
        return float(x)


df.loc[:, "값"] = df["값"].apply(to_float)

tmp = df.dropna()
tmp.info()
tmp

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550 entries, 0 to 2549
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   통계표            2550 non-null   object 
 1   시점             2550 non-null   object 
 2   주기_코드          2550 non-null   object 
 3   행정구역(시군구)별     2550 non-null   object 
 4   행정구역(시군구)별_코드  2550 non-null   object 
 5   항목명            2550 non-null   object 
 6   값              2550 non-null   float64
 7   단위             2550 non-null   object 
 8   주기             2550 non-null   object 
dtypes: float64(1), object(8)
memory usage: 179.4+ KB


,통계표,시점,주기_코드,행정구역(시군구)별,행정구역(시군구)별_코드,항목명,값,단위,주기
0,행정구역(시군구)별 성별 인구수,2019,A,전국,00,총인구수,51849861.0,명,연간
1,행정구역(시군구)별 성별 인구수,2019,A,전국,00,남자인구수,25864816.0,명,연간
2,행정구역(시군구)별 성별 인구수,2019,A,전국,00,여자인구수,25985045.0,명,연간
3,행정구역(시군구)별 성별 인구수,2019,A,서울특별시,11,총인구수,9729107.0,명,연간
4,행정구역(시군구)별 성별 인구수,2019,A,서울특별시,11,남자인구수,4744059.0,명,연간
...,...,...,...,...,...,...,...,...,...
2545,행정구역(시군구)별 성별 인구수,2021,A,제주시,50110,남자인구수,246494.0,명,연간
2546,행정구역(시군구)별 성별 인구수,2021,A,제주시,50110,여자인구수,246602.0,명,연간
2547,행정구역(시군구)별 성별 인구수,2021,A,서귀포시,50130,총인구수,183663.0,명,연간
2548,행정구역(시군구)별 성별 인구수,2021,A,서귀포시,50130,남자인구수,92683.0,명,연간


### 행정동코드

In [ ]:
# 요청 받아오기
res = requests.get("https://grpc-proxy-server-mkvo6j4wsq-du.a.run.app/v1/regcodes?regcode_pattern=*****00000")  # 시군구 (앞 5자리 와일드카드)
# res = requests.get("https://grpc-proxy-server-mkvo6j4wsq-du.a.run.app/v1/regcodes?regcode_pattern=********00")  # 읍면동까지 (앞8자리 와일드카드)
raw = res.json()["regcodes"]

# 음면동만 남김
codes = []
for row in raw:
    spl = row["name"].split(" ")

    for i, sp in enumerate(spl):
        row[f"r{i+1}"] = sp

    codes.append(row)

code = pd.DataFrame(codes)
code.to_csv("행정동코드(시군구까지)_2022.07.23.csv")
code

,code,name,r1,r2,r3
0,1100000000,서울특별시,서울특별시,NaN,NaN
1,1111000000,서울특별시 종로구,서울특별시,종로구,NaN
2,1114000000,서울특별시 중구,서울특별시,중구,NaN
3,1117000000,서울특별시 용산구,서울특별시,용산구,NaN
4,1120000000,서울특별시 성동구,서울특별시,성동구,NaN
...,...,...,...,...,...
272,4888000000,경상남도 거창군,경상남도,거창군,NaN
273,4889000000,경상남도 합천군,경상남도,합천군,NaN
274,5000000000,제주특별자치도,제주특별자치도,NaN,NaN
275,5011000000,제주특별자치도 제주시,제주특별자치도,제주시,NaN


In [ ]:
# code 컬럼을 이용해 시도만 남기기 (!= 로 하면 삭제로 가능)
temp = code.loc[code["code"].str[2:] == "0" * 8, :]  # 시도는 앞 2자리만 있고, 나머지는 0, 시군구는 앞 5자리
temp

,code,name,r1,r2,r3
0,1100000000,서울특별시,서울특별시,NaN,NaN
26,2600000000,부산광역시,부산광역시,NaN,NaN
43,2700000000,대구광역시,대구광역시,NaN,NaN
52,2800000000,인천광역시,인천광역시,NaN,NaN
63,2900000000,광주광역시,광주광역시,NaN,NaN
69,3000000000,대전광역시,대전광역시,NaN,NaN
75,3100000000,울산광역시,울산광역시,NaN,NaN
82,4100000000,경기도,경기도,NaN,NaN
131,4200000000,강원도,강원도,NaN,NaN
150,4300000000,충청북도,충청북도,NaN,NaN


In [ ]:
# 시군구만
temp = code.loc[code["code"].str[2:] != "0" * 8, :]  # 시도 없애기
temp = temp.loc[code["code"].str[5:] == "0" * 5, :]  # 시군구만 남기기
temp.loc[:, "new_code"] = temp["code"].str[:5]
temp

,code,name,r1,r2,r3,new_code
1,1111000000,서울특별시 종로구,서울특별시,종로구,NaN,11110
2,1114000000,서울특별시 중구,서울특별시,중구,NaN,11140
3,1117000000,서울특별시 용산구,서울특별시,용산구,NaN,11170
4,1120000000,서울특별시 성동구,서울특별시,성동구,NaN,11200
5,1121500000,서울특별시 광진구,서울특별시,광진구,NaN,11215
...,...,...,...,...,...,...
271,4887000000,경상남도 함양군,경상남도,함양군,NaN,48870
272,4888000000,경상남도 거창군,경상남도,거창군,NaN,48880
273,4889000000,경상남도 합천군,경상남도,합천군,NaN,48890
275,5011000000,제주특별자치도 제주시,제주특별자치도,제주시,NaN,50110


In [ ]:
temp.info()
temp.loc[temp["r2"].isna(), :]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261 entries, 1 to 276
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   code      261 non-null    object
 1   name      261 non-null    object
 2   r1        261 non-null    object
 3   r2        260 non-null    object
 4   r3        32 non-null     object
 5   new_code  261 non-null    object
dtypes: object(6)
memory usage: 14.3+ KB


,code,name,r1,r2,r3,new_code
81,3611000000,세종특별자치시,세종특별자치시,NaN,NaN,36110


In [ ]:
# 코드 합치는 방법 (left merge)
tmp2 = temp.rename({"new_code": "행정구역(시군구)별_코드"}, axis=1)

merged = pd.merge(tmp, tmp2[["행정구역(시군구)별_코드", "name"]], how="left", on="행정구역(시군구)별_코드")
merged = merged.loc[merged["name"].dropna().index, :]  # 코드 2자리라 name이 안 들어온 것들 drop
merged.info()
merged

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2349 entries, 6 to 2549
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   통계표            2349 non-null   object 
 1   시점             2349 non-null   object 
 2   주기_코드          2349 non-null   object 
 3   행정구역(시군구)별     2349 non-null   object 
 4   행정구역(시군구)별_코드  2349 non-null   object 
 5   항목명            2349 non-null   object 
 6   값              2349 non-null   float64
 7   단위             2349 non-null   object 
 8   주기             2349 non-null   object 
 9   name           2349 non-null   object 
dtypes: float64(1), object(9)
memory usage: 201.9+ KB


,통계표,시점,주기_코드,행정구역(시군구)별,행정구역(시군구)별_코드,항목명,값,단위,주기,name
6,행정구역(시군구)별 성별 인구수,2019,A,종로구,11110,총인구수,151290.0,명,연간,서울특별시 종로구
7,행정구역(시군구)별 성별 인구수,2019,A,종로구,11110,남자인구수,73746.0,명,연간,서울특별시 종로구
8,행정구역(시군구)별 성별 인구수,2019,A,종로구,11110,여자인구수,77544.0,명,연간,서울특별시 종로구
9,행정구역(시군구)별 성별 인구수,2019,A,중구,11140,총인구수,126171.0,명,연간,서울특별시 중구
10,행정구역(시군구)별 성별 인구수,2019,A,중구,11140,남자인구수,61910.0,명,연간,서울특별시 중구
...,...,...,...,...,...,...,...,...,...,...
2545,행정구역(시군구)별 성별 인구수,2021,A,제주시,50110,남자인구수,246494.0,명,연간,제주특별자치도 제주시
2546,행정구역(시군구)별 성별 인구수,2021,A,제주시,50110,여자인구수,246602.0,명,연간,제주특별자치도 제주시
2547,행정구역(시군구)별 성별 인구수,2021,A,서귀포시,50130,총인구수,183663.0,명,연간,제주특별자치도 서귀포시
2548,행정구역(시군구)별 성별 인구수,2021,A,서귀포시,50130,남자인구수,92683.0,명,연간,제주특별자치도 서귀포시
